In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import sqlalchemy
import os

from dotenv import load_dotenv
load_dotenv()

import matplotlib as plt
%matplotlib inline

In [2]:
conn = psycopg2.connect(
    user=os.getenv("POSTGRES_USER"), 
    password=os.getenv("POSTGRES_PASSWORD"), 
    host="127.0.0.1", 
    port="54320", 
    database="dcat")

In [3]:
cur = conn.cursor()

In [4]:
print ( conn.get_dsn_parameters(),"\n")

{'user': 'dcat', 'dbname': 'dcat', 'host': '127.0.0.1', 'port': '54320', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [5]:
cur.execute("SELECT version();")
record = cur.fetchone()
print("You are connected to - ", record,"\n")

You are connected to -  ('PostgreSQL 12.0 (Debian 12.0-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',) 



In [ ]:
cur.execute("drop index if exists idx_tbl_er_c1")
cur.execute("drop index if exists idx_tbl_er_c2")

# Node details table

In [6]:
cur.execute("drop table if exists tbl_detail")
cur.execute("""
create table tbl_detail 
(  
  id int primary key,    -- ID  
  info jsonb,            -- description  
  crt_time timestamp     -- Creation time  
);  
""")

In [7]:
sql = "select * from tbl_detail"
df = sqlio.read_sql_query(sql, conn)
df.head()

,id,info,crt_time


# Relationship description/attributes

In [8]:
cur.execute("drop table if exists tbl_er_desc")
cur.execute("""
create table tbl_er_desc  -- Relationship description  
(  
  id int2 primary key,    -- Relationship ID  
  info text  -- Description  
);
""")

In [ ]:
cur.execute("insert into tbl_er_desc values (1,"");"))

In [9]:
sql = "select * from tbl_er_desc"
df = sqlio.read_sql_query(sql, conn)
df.head()

,id,info


# Relationships/Edges 

In [10]:
cur.execute("drop table if exists tbl_er")
cur.execute("""
create table tbl_er     -- What is relationship between ID1 to ID2      
(  
  c1 int references tbl_detail(id),    
  c2 int references tbl_detail(id),    
  prop int2[],  -- Multiple relationships may exist. Therefore, we use array storage. This is an edge. 
  crt_time timestamp,  
  check (c1<>c2),  
  unique (c1,c2)  
  -- FOREIGN KEY (EACH ELEMENT OF prop) REFERENCES tbl_er_desc(id)  -- Array foreign key, which is supported for PostgreSQL 11. The performance is good  
);  
""")

In [11]:
sql = "select * from tbl_er"
df = sqlio.read_sql_query(sql, conn)
df.head()

,c1,c2,prop,crt_time


In [12]:
cur.execute("drop index if exists idx_tbl_er_c1")
cur.execute("create index idx_tbl_er_c1 on tbl_er(c1)")

In [13]:
cur.execute("drop index if exists idx_tbl_er_c2")
cur.execute("create index idx_tbl_er_c2 on tbl_er(c2)")

In [18]:
conn.commit()